In [1]:
#importing relevant libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error, r2_score

In [2]:
#loading dataset
d1 = pd.read_csv('D:/snajeet/Sanjeet/test.tsv', encoding= 'unicode_escape', sep='\t')

In [3]:
#loading zips
d2 = pd.read_csv('D:/snajeet/Sanjeet/uszips.csv')

In [4]:
#d2 has a lot of unnecessary information. Hence selecting necessary information anly
d2  = d2[['zip', 'lat', 'lng']]

In [5]:
d1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500000 entries, 0 to 2499999
Data columns (total 19 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   b2c_c2c                    object 
 1   seller_id                  int64  
 2   declared_handling_days     float64
 3   acceptance_scan_timestamp  object 
 4   shipment_method_id         int64  
 5   shipping_fee               float64
 6   carrier_min_estimate       int64  
 7   carrier_max_estimate       int64  
 8   item_zip                   object 
 9   buyer_zip                  object 
 10  category_id                int64  
 11  item_price                 float64
 12  quantity                   int64  
 13  payment_datetime           object 
 14  delivery_date              float64
 15  weight                     int64  
 16  weight_units               int64  
 17  package_size               object 
 18  record_number              int64  
dtypes: float64(4), int64(9), object(6)
memory 

In [6]:
d1.columns

Index(['b2c_c2c', 'seller_id', 'declared_handling_days',
       'acceptance_scan_timestamp', 'shipment_method_id', 'shipping_fee',
       'carrier_min_estimate', 'carrier_max_estimate', 'item_zip', 'buyer_zip',
       'category_id', 'item_price', 'quantity', 'payment_datetime',
       'delivery_date', 'weight', 'weight_units', 'package_size',
       'record_number'],
      dtype='object')

In [7]:
#converting item_zip from object to a number
items = []
for item in d1['item_zip']:
    #checking if the item is a string
    if isinstance(item, str):
        #remove comma if any
        item = item.replace(",", "")
        #parsing length to 5
        item = item[:5]
        #spliting on delimeter if any
        item = item.split("-")[0]
    #converting into int
    try:
        item = int(item)
    except:
        item = 0
    items.append(item)


d1['item_zip'] = items

In [8]:
#converting item_zip from object to a number
items = []
for item in d1['buyer_zip']:
    #checking if the item is a string
    if isinstance(item, str):
        #remove comma if any
        item = item.replace(",", "")
        #parsing length to 5
        item = item[:5]
        #spliting on delimeter if any
        item = item.split("-")[0]
    #converting into int
    try:
        item = int(item)
    except:
        item = 0
    items.append(item)


d1['buyer_zip'] = items

In [9]:
#parsing the payment_datetime
items = []
for item in d1['payment_datetime']:
    item = item.split(" ")[0]
    items.append(item)

d1['payment_datetime'] = items

In [10]:
#parsing the acceptance_scan_timestamp
items = []
for item in d1['acceptance_scan_timestamp']:
    item = item.split(" ")[0]
    items.append(item)

d1['acceptance_scan_timestamp'] = items

In [11]:
#meerging process is two step same as in the training notebook
merge_1 = pd.merge(d2, d1, left_on = "zip", right_on = "item_zip", how = 'right')

In [12]:
#renaming the columns 
merge_1.rename(columns={'lng': 'item_lng', 'lat': 'item_lat'}, inplace=True)

In [13]:
merge_2 = pd.merge(d2, merge_1, left_on = "zip", right_on = "buyer_zip", how = 'right')

In [14]:
#renaming the columns 
merge_2.rename(columns={'lng': 'buyer_lng', 'lat': 'buyer_lat'}, inplace=True)

In [15]:
df = merge_2

In [16]:
#checking null values
df.isnull().sum()

zip_x                          29834
buyer_lat                      29834
buyer_lng                      29834
zip_y                          54688
item_lat                       54688
item_lng                       54688
b2c_c2c                            0
seller_id                          0
declared_handling_days         14640
acceptance_scan_timestamp          0
shipment_method_id                 0
shipping_fee                       0
carrier_min_estimate               0
carrier_max_estimate               0
item_zip                           0
buyer_zip                          0
category_id                        0
item_price                         0
quantity                           0
payment_datetime                   0
delivery_date                2500000
weight                             0
weight_units                       0
package_size                       0
record_number                      0
dtype: int64

In [17]:
#filling numeric value columns with forward filling technique
df = df.fillna(method="ffill")

In [18]:
#for filling object columns
df = df.mask(df=='nan', None).ffill()

In [19]:
#Finding the distance between the item and the buyer on the basis of longitudes and latitudes
buyer_lng=np.radians(df['buyer_lng'])
item_lng=np.radians(df['item_lng'])
buyer_lat=np.radians(df['buyer_lat'])
item_lat=np.radians(df['item_lat'])

dlon = buyer_lng - item_lng
dlat = buyer_lat - item_lat

# using haversine formula
a = np.sin(dlat/2)**2 + np.cos(buyer_lat) * np.cos(item_lat) * np.sin(dlon/2)**2

c = 2 * np.arcsin(np.sqrt(a)) 

# Radius of earth in kilometers is 6371
km = 6371* c

df['distance'] = km

In [20]:
#To calculate delay in payment acceptance
df['del_to_pay'] = ((pd.to_datetime(df['acceptance_scan_timestamp']) - pd.to_datetime(df['payment_datetime'])).astype('timedelta64[D]')).astype(int)

In [21]:
#removing innecessary columns
df = df[['b2c_c2c', 'declared_handling_days', 'shipment_method_id', 'shipping_fee',
       'carrier_min_estimate', 'carrier_max_estimate',
       'category_id', 'item_price', 'quantity',
       'weight', 'weight_units', 'package_size',
       'distance', 'del_to_pay']]

In [22]:
df['b2c_c2c'].value_counts()

B2C    1722898
C2C     777102
Name: b2c_c2c, dtype: int64

In [23]:
# converting 'C2C','B2C' into 1,2
df['b2c_c2c'].replace(to_replace=['C2C','B2C'],value=[1,2],inplace=True)

In [24]:
#converting package size to numbers
df['package_size'].value_counts()

PACKAGE_THICK_ENVELOPE    2153288
NONE                       146792
LETTER                     137274
LARGE_ENVELOPE              33863
LARGE_PACKAGE               28783
Name: package_size, dtype: int64

In [25]:
# Making the package size from str to int
df['package_size'].replace(to_replace=['PACKAGE_THICK_ENVELOPE','NONE','LETTER','LARGE_ENVELOPE','LARGE_PACKAGE'],value=[2,3,1,4,5],inplace=True)

Sklearn models

In [26]:
#loading the model
from joblib import load
#change the model name as per requirement
reg = load('ridge_all.joblib') 
days = reg.predict(df)

In [27]:
#ensuring that the delivery days are positive
for i in range(len(days)):
    if days[i] <0:
        days[i] = 0
    else:
        pass
        days[i] = int(days[i])

In [29]:
#making a new dataframe for output
df_out = d1[['record_number', 'payment_datetime']]

In [30]:
df_out['days']=days

C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_8704/3839791159.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_out['days']=days


In [31]:
#adding delivery days to payment date to find delivery date
df_out['delivery_date'] = pd.to_datetime(df_out["payment_datetime"]) + pd.to_timedelta(df_out['days'], unit='D')

C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_8704/714114002.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_out['delivery_date'] = pd.to_datetime(df_out["payment_datetime"]) + pd.to_timedelta(df_out['days'], unit='D')


In [32]:
df_out.head(5)

,record_number,payment_datetime,days,delivery_date
0,15000001,2019-11-13,5.0,2019-11-18
1,15000002,2019-11-26,5.0,2019-12-01
2,15000003,2019-12-02,5.0,2019-12-07
3,15000004,2019-12-12,5.0,2019-12-17
4,15000005,2019-12-18,5.0,2019-12-23


In [33]:
#keeping only required columns in the output dataframe
df_out_1 = df_out[['record_number', 'delivery_date']]

In [34]:
#saving the file as tsv
df_out_1.to_csv('adaboost.tsv', sep = '\t', header=False, index = False)

Tensorflow models

In [31]:
from tensorflow.keras.models import load_model
adamax = load_model("adamax.h5")

In [33]:
days =  adamax.predict(df)

In [34]:
#ensureing that all the days are positive
for i in range(len(days)):
    if days[i] <0:
        days[i] = 0
    else:
        days[i] = int(days[i])

In [36]:
df_out['days']=days

C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_14664/3839791159.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_out['days']=days


In [37]:
#adding delivery days to payment date to find delivery date
df_out['delivery_date'] = pd.to_datetime(df_out["payment_datetime"]) + pd.to_timedelta(df_out['days'], unit='D')

C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_14664/714114002.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_out['delivery_date'] = pd.to_datetime(df_out["payment_datetime"]) + pd.to_timedelta(df_out['days'], unit='D')


In [40]:
#removing unnecessary columns
df_out_2 = df_out[['record_number', 'delivery_date']]

In [41]:
#saving the file as tsv
df_out_2.to_csv('adamax.tsv', sep = '\t', header=False, index = False)